# Everything is an Object
On *nix type operating systems, "everything is a file". Meaning the operating system exposes a set of programming calls that treat all the various parts of an operating system as though they were files. Network sockets, processes and threads, and areas of live memory are files with relative locations in a tree structure, that can be read from and written to, and have read and write permissions applied for certain users and groups.

On Windows, everything is instead, an Object. This means everything is represented in memory as an object data structure, with properties and members, and methods that collectively provide an API to that operating system component.

The **object manager** is the component of the Windows NTOSKernel (or kernel for short) that managed these objects, their memory allocation, and lifetimes.

The kernel maintains a list of _types_ of object it supports, which can be examined with the `Get-NtType` cmdlet.

In [2]:
Get-NtType | Select -First 20


Name
----
Type
Directory
SymbolicLink
Token
Job
Process
Thread
Partition
UserApcReserve
IoCompletionReserve
ActivityReference
ProcessStateChange
ThreadStateChange
CpuPartition
PsSiloContextPaged
PsSiloContextNonPaged
DebugObject
Event
Mutant
Callback



# Object Manager Namespace
The Object Manager Namespace or OMNS for short, is a filesystem like structure hidden in the background of the operating system. Calling it filesystem like is not just descrptive - its literally made out of `Directory` type objects, and can be interacted with like a disc backed filesystem. The directory objects contain instances of kernel objects as they exist in memory, which can be interacted with like files in the directory.

The `NtObjectManager` module contains a PSdrive provider so we can use PowerShell to traverse the OMNS like any other drive.

In [18]:
Get-ChildItem NtObject:\ | Sort Name


Name
----                                                                                               
ArcName                                                                                            
BaseNamedObjects                                                                                   
BindFltPort                                                                                        
Callback                                                                                           
CLDMSGPORT                                                                                         
clfs                                                                                               
Container_Microsoft.WidgetsPlatformRuntime_1.6.1.0_x64__8wekyb3d8bbwe-S-1-5-21-2776884319-30901428…
Container_Microsoft.YourPhone_1.24121.85.0_x64__8wekyb3d8bbwe-S-1-5-21-2776884319-3090142823-33783…
Container_MicrosoftWindows.Client.WebExperience_524.34401.20.0_x64__cw5n1h2txyewy-S-1-5-21-277

In [24]:
Get-ChildItem NtObject:\Dfs | Select -Property *


PSPath               : NtObjectManager\NtObjectManager::nt:\Dfs
PSParentPath         : NtObjectManager\NtObjectManager::nt:
PSChildName          : Dfs
PSDrive              : NtObject
PSProvider           : NtObjectManager\NtObjectManager
PSIsContainer        : False
Name                 : Dfs
TypeName             : SymbolicLink
IsDirectory          : False
IsDevice             : False
IsSymbolicLink       : True
RelativePath         : Dfs
SecurityDescriptor   : O:BAG:SYD:(A;;CCRC;;;WD)(A;;CCSDRCWDWO;;;SY)(A;;CCSDRCWDWO;;;BA)(A;;CCRC;;;R
                       C)
SymbolicLinkTarget   : \Device\DfsClient
MaximumGrantedAccess : Query, ReadControl
DeviceType           : UNKNOWN
Characteristics      : None



In [26]:
Get-Item NtObject:\Device\DfsClient | Select -Property *


PSPath               : NtObjectManager\NtObjectManager::nt:\Device\DfsClient
PSParentPath         : NtObjectManager\NtObjectManager::nt:\Device
PSChildName          : DfsClient
PSDrive              : NtObject
PSProvider           : NtObjectManager\NtObjectManager
PSIsContainer        : False
Name                 : DfsClient
TypeName             : Device
IsDirectory          : False
IsDevice             : True
IsSymbolicLink       : False
RelativePath         : Device\DfsClient
SecurityDescriptor   : O:BAG:SYD:(A;;FA;;;S-1-5-80-719998295-2833700043-1566817583-4093942769-14140
                       26312)(A;;FA;;;SY)(A;;FA;;;BA)(A;;FX;;;WD)
SymbolicLinkTarget   : 
MaximumGrantedAccess : Execute, ReadAttributes, ReadControl, Synchronize
DeviceType           : DISK_FILE_SYSTEM
Characteristics      : RemoteDevice, SecureOpen

